In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import logging
import uuid
from pathlib import Path
from datetime import datetime, timedelta
from typing import Optional, Dict, List, Tuple
from client import LocalLLMClient, create_client
from news_function import analyze_news_single  # 复用单条分析函数


# --------------------------
# 1. 基础配置
# --------------------------
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - 批量拆分分析 - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("batch_split_analysis.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# 结果存储目录（自动创建）
OUTPUT_DIR = Path("batch_split_results")
OUTPUT_DIR.mkdir(exist_ok=True)


# --------------------------
# 2. 工具函数：写入或追加数据到Parquet文件
# --------------------------
def write_or_append_parquet(df: pd.DataFrame, file_path: str):
    """
    将DataFrame写入Parquet文件，如文件已存在则追加
    
    参数：
    - df: 要写入或追加的数据
    - file_path: 目标Parquet文件路径
    """
    table = pa.Table.from_pandas(df)
    
    # 如果文件不存在，创建新文件；否则追加
    if not Path(file_path).exists():
        pq.write_table(table, file_path)
    else:
        # 读取现有schema并追加
        existing_table = pq.read_table(file_path)
        with pq.ParquetWriter(file_path, existing_table.schema) as writer:
            writer.write_table(table)


# --------------------------
# 3. 核心函数：生成双表数据（单日结果）
# --------------------------
def _process_daily_results(daily_results: List[Dict]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """处理单日结果，生成行业裁决表和个股推荐表的DataFrame"""
    industry_records: List[Dict] = []
    stock_records: List[Dict] = []

    for result in daily_results:
        # 基础元数据
        base_meta = {
            "original_news_id": result["original_news_id"],
            "original_news_date": result["original_news_date"],
            "original_news_title": result["original_news_title"],
            "analysis_status": result["status"],
            "error_msg": result["error_msg"],
            "log_path": result["log_path"],
            "analysis_time": result["analysis_batch_time"]
        }

        # 处理分析失败的记录
        if result["status"] == "failed":
            industry_records.append({
                **base_meta,
                "analysis_id": f"ana_{uuid.uuid4().hex[:8]}",
                "industry": "",
                "impact_direction": "",
                "confidence_score": 0,
                "comprehensive_reason": ""
            })
            continue

        # 处理分析成功的记录
        for industry in result["ruled_industries"]:
            current_analysis_id = f"ana_{uuid.uuid4().hex[:8]}"

            # 行业裁决主表记录
            industry_records.append({
                **base_meta,
                "analysis_id": current_analysis_id,
                "industry": industry.get("industry", ""),
                "impact_direction": industry.get("impact", ""),
                "confidence_score": industry.get("confidence", 0),
                "comprehensive_reason": industry.get("comprehensive_reason", "")
            })

            # 个股推荐副表记录
            for rank, stock in enumerate(industry.get("stocks", []), 1):
                stock_records.append({
                    "analysis_id": current_analysis_id,
                    "stock_name": stock.get("name", ""),
                    "stock_code": stock.get("code", ""),
                    "recommendation_reason": stock.get("reason", ""),
                    "stock_rank": rank
                })

    # 转换为DataFrame并调整字段顺序
    industry_df = pd.DataFrame(industry_records)[[
        "analysis_id", "original_news_id", "original_news_date", "original_news_title",
        "industry", "impact_direction", "confidence_score", "comprehensive_reason",
        "analysis_status", "error_msg", "log_path", "analysis_time"
    ]]

    stock_df = pd.DataFrame(stock_records)[[
        "analysis_id", "stock_name", "stock_code", "recommendation_reason", "stock_rank"
    ]]

    return industry_df, stock_df


# --------------------------
# 4. 批量分析主函数（按日期保存）
# --------------------------
def batch_analyze_split_parquet(
    input_parquet: str,
    news_date: str,
    debate_rounds: int = 2,
    llm_client: Optional[LocalLLMClient] = None,** llm_kwargs
) -> Tuple[str, str]:
    """
    批量分析Parquet新闻，按日期完成后统一保存结果
    
    参数：
    - input_parquet: 输入新闻Parquet路径
    - news_date: 新闻日期
    - debate_rounds: 辩论轮次
    - llm_client: 已实例化的LLM客户端
    - **llm_kwargs: 创建LLM客户端的参数
    
    返回：
    - 行业裁决表路径, 个股推荐表路径
    """
    # 1. 初始化LLM客户端
    if not llm_client:
        try:
            llm_client = create_client(** llm_kwargs)
            logger.info("LLM客户端初始化成功")
        except Exception as e:
            raise Exception(f"LLM客户端初始化失败：{str(e)}")

    # 2. 读取并处理输入Parquet
    try:
        logger.info(f"读取输入Parquet：{input_parquet}")
        input_df = pd.read_parquet(input_parquet)
        required_cols = ["id", "date", "title", "content"]
        missing_cols = [col for col in required_cols if col not in input_df.columns]
        if missing_cols:
            raise Exception(f"输入缺少字段：{', '.join(missing_cols)}")
        
        # 筛选出指定日期的新闻并排序
        input_df_filtered = input_df[input_df["date"] == news_date].sort_values(by="id", ascending=True).reset_index(drop=True)
        total_news = len(input_df_filtered)
        logger.info(f"读取 {total_news} 条 {news_date} 的新闻")
    except Exception as e:
        raise Exception(f"读取输入失败：{str(e)}")

    # 如果当天没有新闻，直接返回空路径
    if total_news == 0:
        logger.info(f"{news_date} 没有新闻需要分析")
        return "", ""

    # 3. 准备输出文件路径（带日期标识）
    date_suffix = news_date.replace("-", "")
    industry_path = OUTPUT_DIR / f"industry_verdict_{date_suffix}.parquet"
    stock_path = OUTPUT_DIR / f"stock_recommendation_{date_suffix}.parquet"
    
    # 4. 处理当日所有新闻
    daily_results: List[Dict] = []
    processed_count = 0

    for idx, row in input_df_filtered.iterrows():
        news_id = str(row["id"])
        logger.info(f"分析第 {idx+1}/{total_news} 条新闻（ID：{news_id}）")
        
        # 调用单条分析函数
        single_result = analyze_news_single(
            news_title=str(row["title"]),
            news_content=str(row["content"]),
            debate_rounds=debate_rounds,
            llm_client=llm_client
        )

        # 补充原始新闻元数据
        single_result.update({
            "original_news_id": news_id,
            "original_news_date": str(row["date"]),
            "original_news_title": str(row["title"]),
            "analysis_batch_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })
        daily_results.append(single_result)
        processed_count += 1

    # 5. 当日新闻分析完成后，统一保存结果
    logger.info(f"{news_date} 新闻分析完成，共处理 {processed_count} 条，开始保存结果...")
    
    # 处理当日结果
    daily_industry_df, daily_stock_df = _process_daily_results(daily_results)
    
    # 保存结果（如文件已存在则追加）
    write_or_append_parquet(daily_industry_df, str(industry_path))
    write_or_append_parquet(daily_stock_df, str(stock_path))
    
    logger.info(f"{news_date} 结果保存完成，行业裁决表：{industry_path}，个股推荐表：{stock_path}")

    return str(industry_path), str(stock_path)


# --------------------------
# 5. 使用示例
# --------------------------
if __name__ == "__main__":
    # 配置参数
    INPUT_PARQUET = "../data/stock_daily_cctvnews.parquet"  # 输入新闻路径
    DEBATE_ROUNDS = 2  # 辩论轮次
    LLM_KWARGS = {
        # "model": "gpt-4-turbo",  # 实际使用时填写LLM参数
        # "api_key": "your_api_key"
    }

    # 获取20250101至今以来的日期列表，形成yyyy-mm-dd的格式
    start_date = datetime(2025, 1, 1)
    end_date = datetime.now()
    date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    date_list = [date.strftime("%Y-%m-%d") for date in date_list]

    # 依次遍历date_list ，对每个日期进行分析
    for news_date in date_list:
        try:
            # 执行批量拆分分析
            industry_table_path, stock_table_path = batch_analyze_split_parquet(
                input_parquet=INPUT_PARQUET,
                news_date=news_date,
                debate_rounds=DEBATE_ROUNDS,** LLM_KWARGS
            )
            
            if industry_table_path and stock_table_path:
                # 打印结果
                print("="*70)
                print(f"{news_date} 分析完成！")
                print(f"输入文件：{INPUT_PARQUET}")
                print(f"行业裁决表：{industry_table_path}")
                print(f"个股推荐表：{stock_table_path}")
                print(f"提示：可通过 'analysis_id' 字段关联两张表")
                print("="*70)
        except Exception as e:
            print(f"{news_date} 分析失败：{str(e)}")
            logger.error(f"{news_date} 分析失败：{str(e)}")


2025-10-11 16:55:48,215 - INFO - LLM客户端初始化成功
2025-10-11 16:55:48,216 - INFO - 读取输入Parquet：../data/stock_daily_cctvnews.parquet


2025-10-11 16:55:48,375 - INFO - 读取 12 条新闻，已按id逆序
2025-10-11 16:55:48,376 - INFO - 分析第 1/12 条新闻（ID：114300）
2025-10-11 16:55:48,377 - INFO - 开始分析单条新闻（ID：news_single_20251011165548_822，标题：《求是》杂志发表习近平总书记重要文章《以中国式现代化全面推进...）
2025-10-11 16:55:48,377 - INFO - 请求模型: qwen3:30b-a3b-instruct-2507-q4_K_M
2025-10-11 16:55:48,378 - INFO - 用户消息: 标题：《求是》杂志发表习近平总书记重要文章《以中国式现代化全面推进强国建设、民族复兴伟业》
内容：央视网消息
（新闻联播）：2025年1月1日出版的第1期《求是》杂志发表中共中央总书记、国家主席、中央军委主席习近平的重要文章《以中国式现代化全面推进强国建设、民族复兴伟业》。
文章强调，概括提出并深入阐述中国式现代化理论，是党的二十大的一个重大理论创新，是科学社会主义的最新重大成果。
文章指出，中国式现代化是我们党领导人民长期探索和实践的重大成果。党的十八大以来，我们党在已有基础上继续前进，坚持问题导向，围绕解决现代化建设中存在的突出矛盾和问题，全面深化改革，不断实现理论和实践上的创新突破，成功推进和拓展了中国式现代化。我们在认识上不断深化，在战略上不断完善，在实践上不断丰富，推动党和国家事业取得历史性成就、发生历史性变革，为中国式现代化提供了更为完善的制度保证、更为坚实的物质基础、更为主动的精神力量。
文章指出，党的领导直接关系中国式现代化的根本方向、前途命运、最终成败。党的二十大报告明确指出：“中国式现代化，是中国共产党领导的社会主义现代化”。这是对中国式现代化定性的话，是管总、管根本的。党的领导决定中国式现代化的根本性质。党的领导确保中国式现代化锚定奋斗目标行稳致远。党的领导激发建设中国式现代化的强劲动力。党的领导凝聚建设中国式现代化的磅礴力量。
文章指出，中国式现代化是强国建设、民族复兴的康庄大道。中国式现代化既有各国现代化的共同特征，更有基于自己国情的鲜明特色。第一，人口规模巨大的现代化。第二，全体人民

KeyboardInterrupt: 